---
Ce programme nous permet de traiter les données Events de Stats Bomb pour en extraire :
1) Les centres ainsi qu'un certain nombre d'events les suivant (ce ne nombre est défini à l'aide de la variable nb_events_suivant)
2) Les positions de ces centres et de leur zone de réception
3) Pour ces events suivant les centres, on repère les tirs et les buts inscrits par l'équipe qui a centré
4) Les positions de ces tirs ainsi que des endroits ou ils se sont terminés
5) Des informations liées à ces events telles que la date du match, la journée, les équipes à domicile et à l'extérieur, la saison, la compétition ainsi que le "match_id" Stats Bomb

Grâce à ces données, nous pourrons afficher sur un terrain, via l'application Web Streamlit :
- Des cartes de chaleurs "heatmaps" des zones de centre et de réception de centre
- Des flèches représentant la trajectoire des tirs survenus dans les "nb_events_suivant" events suivant les centres
- Les zones ou se terminent les tirs dans la cage du gardien
---

## 1) Importation des bibliothèques

In [1]:
import pandas as pd
import sqlite3

---

## 2) Connexion à la BDD

In [2]:
connect = sqlite3.connect("../../Databases/raw-database.db")
cursor = connect.cursor()

---

## 3) Importation des données events

In [3]:
colonnes = ["x_loc", "y_loc", "match_id_SB", "type", "team_id_SB", "minute", "shot_outcome", "pass_type", "period",
            "pass_cross", "index_event", "x_shot", "y_shot", "z_shot", "x_pass", "y_pass", "pass_body_part", "player_id_SB"]
colonnes = ", ".join(colonnes)
req = cursor.execute(f"SELECT {colonnes} FROM events")
res = req.fetchall()
desc = req.description
events = pd.DataFrame(res)
events.columns = [i[0] for i in desc]

In [4]:
# Aperçu des données
events.head(2)

,x_loc,y_loc,match_id_SB,type,team_id_SB,minute,shot_outcome,pass_type,period,pass_cross,index_event,x_shot,y_shot,z_shot,x_pass,y_pass,pass_body_part,player_id_SB
0,NaN,NaN,3894037,Starting XI,147,0,None,None,1,NaN,1,NaN,NaN,NaN,NaN,NaN,None,NaN
1,NaN,NaN,3894037,Starting XI,156,0,None,None,1,NaN,2,NaN,NaN,NaN,NaN,NaN,None,NaN


Avant de se lancer dans la suite du traitement, nous allons modifier l'index "index_event", qui n'identifie plus les events de manière unique.  
En effet, de base, cet index permet d'identifier de manière unique l'ensemble des events d'un match, de manière chronologique : l'event d'un match ayant pour "index_event" 1 correspond au premier event référencé de ce match.  
Cependant, étant donné que la table "events" contient les events de plusieurs matchs, cet index "index_event" n'est plus unique.  
Nous pouvons le constater grâce à la cellulle suivante :

In [5]:
events.index_event.is_unique

False

Nous allons donc re-créer cette variable "index_event", afin qu'elle identifie de nouveau les events de manière unique, et par ordre chronologique.  
L'ordre chronologique est assuré par le fait que dans le programme d'importation, nous avons trié les events par "match_id_SB" et en fonction de leur "index_event"

In [6]:
# Réindexation des events
events["index_event"] = range(len(events))

---
Maintenant que nous avons importé les données Events, nous allons extraire les events correspondant à des centres.  
Nb : nous avons préféré ne pas analyser les centres qui sont des corners

## 4) Récupération des events liés à des centres

In [7]:
# Création d'un dataframe ne comprenant que les events liés à des centres (hormis les corners)
centres = events[(events.pass_cross == 1) & (events.pass_type != "Corner")]

---
Dans la prochaine section, nous allons récupérer les events suivant ces centres

## 5) Extraction des events suivant les centres

In [8]:
# Nous décidons de fixer à 5 le nombre d'events suivant les centres à garder
nb_events_suivant = 5

Nous souhaitons donc récupérer les 5 events suivant chaque centre.  
Pour ce faire, nous allons donc utiliser la colonne "index_event".  
Exemple : si nous souhaitons récupérer les 5 events suivant un centre ayant pour "index_event" 3, alors nous devrons extraire les events ayant pour "index_event" 4, 5, 6, 7 et 8.  

Cependant, il faut faire attention, car plusieurs cas peuvent se présenter et poser problème.  
Par exemple, s'il y a un autre centre dans les 5 events, ou si le centre correspond au dernier event d'une mi-temps, et donc les 5 events suivant ce centre se sont déroulés lors d'une autre période/match et il est donc incohérent des les analyser.

### a) Création du dataframe

Dans un premier temps, nous allons créer un "patron" du dataframe qui contiendra les events liés aux centres et les 5 events les suivant (même s'ils se sont déroulés lors d'une autre mi-temps ou lors d'un autre match).  
Ce dataframe aura donc une taille de 6 fois le nombre de centres.  


Ce dataframe contiendra 5 colonnes/informations :
- L'équipe ayant effectué le centre "team_centre", qui sera la même pour le centre et les events suivant ce centre. Cette information nous permettra de savoir si l'équipe liée à un des events suivant le centre est la même que l'équipe qui a effectué le centre associé à cet event
- Le "centre_id", qui sera la même pour le centre et les events suivant ce centre. Il permettra d'associé le centre et les events le suivant
- Les "match_id_SB" et les "period" liés au centre, qui seront eux aussi attribués aux events suivant. Il est donc possible que le "match_id_SB" ou la "period" attribué à un event ne soit pas la même que le réel "match_id_SB"/"period" de cet event. Par exemple, dans le cas ou le centre est le dernier event de la 1ère mi-temps et donc la "period" des 5 events le suivant ne correspondent pas
- La colonne "index_event", qui contiendra les "index_event" de l'ensemble des centres et des events les suivant. Elle permettra d'effectuer la jointure avec les events du dataframe "events", et donc d'extraire les events suivant les centres

In [9]:
centres = pd.DataFrame({"team_centre" : [i for i in centres.team_id_SB for _ in range (nb_events_suivant + 1)],
                    "centre_id" : [i for i in range (1, len(centres) + 1) for _ in range (nb_events_suivant + 1)],
                    "match_id_SB" : [i for i in centres.match_id_SB for _ in range (nb_events_suivant + 1)],
                    "period" : [i for i in centres.period for _ in range (nb_events_suivant + 1)],
                    "index_event" : [j + i for j in centres.index_event for i in range (nb_events_suivant + 1)]})

La méthode "for _ in range (nb_events_suivant + 1)" permet de répéter (nb_events_suivant + 1) fois la valeur itérée par i.  
Par exemple, si nb_events_suivant vaut 5 et i prend les valeurs 1, 2, 3, etc., alors cela donnera :
[1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, etc.].  
C'est cette méthode qui nous permet d'attribuer aux events suivant les centres les informations liées au centre, telles que l'équipe qui l'a effectué, la période ou il a été effectué etc.

In [10]:
# Aperçu des données
centres.head(10)

,team_centre,centre_id,match_id_SB,period,index_event
0,147,1,3894037,1,153
1,147,1,3894037,1,154
2,147,1,3894037,1,155
3,147,1,3894037,1,156
4,147,1,3894037,1,157
5,147,1,3894037,1,158
6,147,2,3894037,1,259
7,147,2,3894037,1,260
8,147,2,3894037,1,261
9,147,2,3894037,1,262


### b) Jointure avec les events du dataframe "events"

Nous pouvons donc effectuer la jointure de ce dataframe avec le dataframe "events", par rapport aux "event_id", "match_id_SB" et "period".  
Ces colonnes utilisées pour la jointure nous permettent de supprimer les cas ou un centre se serait déroulé dans les "nb_events_suivant" derniers events d'une période, et donc de supprimer les events les suivant qui se sont déroulés dans une autre mi-temps.

In [11]:
events_centres = pd.merge(centres, events, on = ["index_event", "match_id_SB", "period"])

### c) Cas ou il y'a un autre centre dans les "nb_events_suivant" events suivant un centre

Nous allons maintenant traiter les cas ou il y'a un centre dans les events suivant un centre.  
Dans ces cas, nous allons uniquement garder le 2ème centre et les events suivant ce dernier.  
Dans le cas ou il y aurait 3 centres qui se suivraient, alors on garderait le 3ème centre et les events suivant ce dernier, et de même pour + de centres se suivant...  

Exemple d'un cas qui pourrait poser problème : on a un centre qui a pour "index_event" 5 et "centre_id" 2, et 3 events après il y a eu un autre centre (qui a donc pour "index_event" 8 et "centre_id" 3).  
Les valeurs actuelles de la colonne "index_event" du dataframe "events_centres" sont donc :  
[5, 6, 7, 8, 9, 10, 8, 9, 10, 11, 12, 13].  
Tandis que les valeurs de la colonne "centre_id" sont :  
[2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3].  
Donc souhaitons donc supprimer les lignes ayant pour "centre_id" 2

Pour repérer ces cas, il suffit donc de repérer les "index_event" dupliqués.  
Pour garder le centre et les events le suivant, nous allons utiliser la commandes "duplicated" de Pandas.  

La fonction "duplicated" permet d'indiqué si une valeur est dupliquée. Elle n'indique pas comme dupliquée la première occurence d'une valeur dupliquée.  
Par exemple, s'il y'a 2 lignes qui contiennent la valeur "ok", alors seulement la deuxième ligne sera marquée comme dupliquée.  
Elle va nous permettre de repérer les "centre_id" dupliqués.

In [12]:
# Récupérations les "centre_id" dupliqués
# Nous regardons les valeurs dupliquées pour les colonnes "team_id_SB" et "index_event".
# En effet, si c'est une autre équipe qui a effectué le centre se déroulant dans les "nb_events_suivant" events suivant le 1er centre,
# alors c'est incohérent de ne pas garder le 1er centre
centre_id_dupl = events_centres.loc[events_centres[["team_id_SB", "index_event"]].duplicated(), "centre_id"].unique() - 1
# Nous ajoutons la commande "unique()" car nous souhaitons seulement connaitre la liste des "centre_id" dupliqués, de manière unique.
# Nous ajoutons "- 1" car nous souhaitons garder les centres suivant d'autres centres, ou plutot supprimer les centres pour lesquels il y'a un autre centre dans les events suivant ces centres.

Cependant, nous n'allons pas supprimer maintenant ces centres.  
En effet, dans le cas ou l'équipe ayant effectué un de ces centres marquerait lors d'un event se déroulant entre ce centre et le centre effectué dans les "nb_events_suivant" events suivant ce centre, alors on souhaiterait garderer les informations liés à ce centre.  
Pour expliquer d'une autre manière ce cas, prenons nb_events_suivant = 10 :  
Si une équipe effectue un centre (qu'on nomme centre 1), qu'elle effectue un autre centre (centre 2) correspondant au 9ème event suivant "centre 1", mais qu'elle marque lors du deuxième event suivant "centre 1", alors nous souhaitons garder "centre 1" ainsi que les events le suivant, même si cette équipe a effectué un autre centre dans les "nb_events_suivant" events suivant "centre 1".  
Nous supprimerons donc ces centres après avoir detécté les centres qui ont amenés un but dans les "nb_events_suivant" events suivant ces centres

In [18]:
events_centres.drop_duplicates("index_event", keep = "last", inplace = True)

In [ ]:
events_centres["Tireur"] = ""

In [ ]:
filtre_shot = (events_centres.type == "Shot") & (events_centres.attacking_team == events_centres.team)
events_centres.loc[filtre_shot, "Tireur"] = events_centres.loc[filtre_shot, "player"]

In [ ]:
events_centres["But"] = "Non"

In [ ]:
filtre_goal = ((events_centres.shot_outcome == "Goal") | (events_centres.type == "Own Goal For")) & (events_centres.attacking_team == events_centres.team)
events_centres.loc[(events_centres.centre_id.isin(events_centres.loc[filtre_goal, "centre_id"])) & (events_centres.pass_cross == 1), "But"] = "Oui"

In [ ]:
events_centres.loc[filtre_goal, "But"] = "Oui"